<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-29 09:19:03
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.37 C
-------------------
Today PnL: 48.14 K (0.35%)
Current PnL: -20.37 L (-13.7%)
CY Booked + Current PnL: -11.33 L (-7.62%)
-------------------
Total profit:  2.75 L
Total loss:  -23.12 L
-------------------
Total Booked + Current PnL: 15.65 L (12.86%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.61%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 81.75 L (59.73%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.61,112352.0,8040.0,8606.0,0.11,7.71,7.66,15.96,9.0,0.93,0.85,25.91,X5K,ATH,METALS
53,MEDANTA,1486.00,12.80,59.0,H-MC,15.04,161078.0,34878.0,11292.0,-0.81,27.64,7.01,36.59,108.0,3.09,1.21,43.67,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-15.08,57.0,H-LC,8.43,169609.0,13432.0,12653.0,-0.24,8.60,7.46,16.71,22.0,1.06,1.28,27.54,X40N,BTT,HEALTHCARE
81,TTKPRESTIG,770.00,98.08,57.0,M-SC,8.18,85327.0,-15450.0,15547.0,0.64,-15.33,18.22,0.09,245.0,-0.99,0.64,12.10,OX40N,NTT,DURABLES
66,SBIN,863.00,-14.20,42.0,M-LC,4.26,212914.0,11434.0,15777.0,0.19,5.68,7.41,13.51,60.0,0.72,1.61,18.99,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,5211.76,-9.59,36.0,H-SC,6.3,102827.0,823.0,43105.0,0.4,0.81,41.92,43.06,122.0,0.02,0.78,18.06,X40N,ATH,MISC


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.40,33.0,H-LC,13.48,240720.0,-20899.0,140123.0,0.29,-7.99,58.21,45.57,7.0,-0.15,1.82,2.43,AR,NTT,ELECTRICAL
33,HCLTECH,1943.91,-0.22,37.0,H-LC,10.22,227917.0,-14007.0,77287.0,0.13,-5.79,33.91,26.15,8.0,-0.18,1.72,7.70,X40,ATH,IT
84,UNITDSPR,1644.00,-15.08,40.0,H-LC,8.75,170716.0,-11043.0,46298.0,0.92,-6.08,27.12,19.39,86.0,-0.24,1.29,1.58,X40N,NTT,BREWERIES
47,JIOFIN,387.00,-0.91,42.0,H-LC,15.25,219205.0,1799.0,51689.0,0.98,0.83,23.58,24.61,48.0,0.03,1.65,56.10,XY24,BTT,FINANCE
69,SIEMENS,7969.85,-4.97,42.0,H-LC,32.68,152610.0,-33485.0,245885.0,0.30,-17.99,161.12,114.13,15.0,-0.14,1.15,12.38,AR,ATH,ELECTRICAL


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NESTLEIND,1377.0,-10.64,56.0,H-LC,2.84,274973.0,9547.0,47240.0,1.09,3.60,17.18,21.40,11.0,0.20,2.07,11.19,XY25,NTT,FMCG
51,LTIM,7230.2,-7.23,47.0,H-LC,2.93,230378.0,-15773.0,94985.0,-0.07,-6.41,41.23,32.18,16.0,-0.17,1.74,27.33,X200,ATH,IT
45,ITC,452.0,-38.34,46.0,H-LC,3.02,198705.0,-1433.0,22315.0,1.36,-0.72,11.23,10.44,4.0,-0.06,1.50,4.81,X40,NTT,FMCG
20,CIPLA,1795.0,-18.75,61.0,H-LC,4.40,216802.0,12302.0,29117.0,0.14,6.02,13.43,20.25,10.0,0.42,1.64,15.27,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.01,58.0,H-LC,7.24,221936.0,-29832.0,152071.0,1.33,-11.85,68.52,48.55,27.0,-0.20,1.67,19.38,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,RELAXO,1176.00,-35.91,52.0,H-SC,8.66,59475.0,-56063.0,91050.0,3.79,-48.52,153.09,30.28,136.0,-0.62,0.45,16.93,X40N,NTT,FOOTWEAR
23,COLPAL,3767.14,0.07,57.0,H-MC,10.61,182169.0,-29572.0,111670.0,3.22,-13.97,61.30,38.77,84.0,-0.26,1.37,8.43,XY25,ATH,FMCG
64,SAMMAANCAP,326.00,-163.64,48.0,M-SC,8.36,72738.0,-29472.0,122862.0,2.76,-28.83,168.91,91.37,208.0,-0.24,0.55,18.08,XY25,NTT,FINANCE
12,BATAINDIA,2096.00,-38.87,41.0,M-SC,9.62,91383.0,-37287.0,82583.0,2.00,-28.98,90.37,35.20,219.0,-0.45,0.69,4.89,X40,NTT,FOOTWEAR
8,AWL,485.00,-65.85,45.0,H-MC,6.86,209798.0,-58649.0,190329.0,1.88,-21.85,90.72,49.05,116.0,-0.31,1.58,7.87,XY24,NTT,FMCG


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BANDHANBNK,400.0,-12.63,40.0,H-SC,8.19,188999.0,-59531.0,274994.0,-1.77,-23.95,145.50,86.70,151.0,-0.22,1.43,26.18,XY24,NTT,BANKS
67,SFL,1287.0,34.22,48.0,M-SC,9.45,177310.0,-84908.0,154738.0,-1.46,-32.38,87.27,26.63,229.0,-0.55,1.34,10.24,XY24,NTT,MISC
28,EASEMYTRIP,26.4,-14.90,31.0,M-SC,29.68,81870.0,-94945.0,175742.0,-1.06,-53.70,214.66,45.70,197.0,-0.54,0.62,0.00,XY24,NTT,TRAVEL
53,MEDANTA,1486.0,12.80,59.0,H-MC,15.04,161078.0,34878.0,11292.0,-0.81,27.64,7.01,36.59,108.0,3.09,1.21,43.67,XY24,NTT,HEALTHCARE
71,SONACOMS,1006.0,-36.69,47.0,M-SC,14.15,78811.0,-22348.0,97237.0,-0.74,-22.09,123.38,74.03,202.0,-0.23,0.59,11.13,AR,BTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,98.08,57.0,M-SC,8.18,85327.0,-15450.0,15547.0,0.64,-15.33,18.22,0.09,245.0,-0.99,0.64,12.10,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,82.03,40.0,H-SC,14.60,114750.0,-27729.0,47254.0,0.16,-19.46,41.18,13.70,163.0,-0.59,0.87,35.44,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.70,50.0,H-SC,3.47,215910.0,-53757.0,90099.0,0.78,-19.93,41.73,13.47,138.0,-0.60,1.63,9.20,XY24,NTT,PAINTS
19,CERA,9475.0,-21.00,39.0,H-SC,3.05,112698.0,-31105.0,57848.0,0.22,-21.63,51.33,18.60,149.0,-0.54,0.85,24.37,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,107.03,38.0,M-SC,2.21,135668.0,-38891.0,38923.0,-0.29,-22.28,28.69,0.02,221.0,-1.00,1.02,18.01,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.76,49.0,H-LC,15.58,291870.0,-54086.0,177953.0,0.37,-15.63,60.97,35.80,1.0,-0.30,2.20,3.40,X40,BTT,IT
43,INFY,2275.00,-18.52,48.0,H-LC,11.69,317455.0,4567.0,167108.0,-0.65,1.46,52.64,54.87,3.0,0.03,2.39,8.21,X40,BTT,IT
45,ITC,452.00,-38.34,46.0,H-LC,3.02,198705.0,-1433.0,22315.0,1.36,-0.72,11.23,10.44,4.0,-0.06,1.50,4.81,X40,NTT,FMCG
87,VBL,671.64,-11.91,47.0,H-LC,8.04,317214.0,1372.0,111279.0,-0.10,0.43,35.08,35.67,5.0,0.01,2.39,14.35,X40N,ATH,FMCG
1,ABB,7934.00,-42.40,33.0,H-LC,13.48,240720.0,-20899.0,140123.0,0.29,-7.99,58.21,45.57,7.0,-0.15,1.82,2.43,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,12.80,59.0,H-MC,15.04,161078.0,34878.0,11292.0,-0.81,27.64,7.01,36.59,108.0,3.09,1.21,43.67,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-6.62,74.0,H-LC,12.84,233230.0,48749.0,21620.0,1.27,26.42,9.27,38.14,38.0,2.25,1.76,39.83,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,12.80,59.0,H-MC,15.04,161078.0,34878.0,11292.0,-0.81,27.64,7.01,36.59,108.0,3.09,1.21,43.67,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,9.32,59.0,H-MC,4.05,188630.0,23960.0,44007.0,0.33,14.55,23.33,41.28,89.0,0.54,1.42,23.69,X40N,ATH,AC
88,VOLTAS,1918.49,-2.76,56.0,H-MC,6.54,203895.0,12153.0,83882.0,0.89,6.34,41.14,50.08,99.0,0.14,1.54,13.57,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178059.0,27146.0,30163.0,-0.08,17.99,16.94,37.97,93.0,0.90,1.34,47.84,AR,ATH,AUTO
24,DABUR,735.00,-6.54,54.0,H-MC,3.78,203229.0,6201.0,83426.0,1.41,3.15,41.05,45.49,102.0,0.07,1.53,16.60,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178059.0,27146.0,30163.0,-0.08,17.99,16.94,37.97,93.0,0.90,1.34,47.84,AR,ATH,AUTO
40,INDIAMART,4850.92,-51.47,47.0,H-SC,12.45,135669.0,12333.0,121424.0,-0.69,10.00,89.50,108.45,119.0,0.10,1.02,34.59,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.68,47.0,M-SC,5.67,96975.0,5476.0,73274.0,-0.28,5.99,75.56,86.07,223.0,0.07,0.73,40.83,XR,NTT,DURABLES
90,WIPRO,420.00,-11.98,50.0,M-LC,7.72,154783.0,3838.0,105624.0,-0.40,2.54,68.24,72.51,53.0,0.04,1.17,8.72,XR,NTT,IT
59,RAJOOENG,143.33,-33.37,37.0,H-SC,10.02,101100.0,-1400.0,42229.0,-0.29,-1.37,41.77,39.83,114.0,-0.03,0.76,0.10,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-45.67,62.0,L-SC,26.10,81542.0,-32007.0,72059.0,-0.17,-28.19,88.37,35.27,268.0,-0.44,0.62,99.58,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.69,36.0,H-SC,16.51,86838.0,-14125.0,108652.0,-0.21,-13.99,125.12,93.63,148.0,-0.13,0.65,29.34,SR,ATH,CHEMICALS
52,MASFIN,398.61,-19.60,45.0,H-SC,12.70,91500.0,-6480.0,28081.0,-0.51,-6.61,30.69,22.05,152.0,-0.23,0.69,33.00,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.61,112352.0,8040.0,8606.0,0.11,7.71,7.66,15.96,9.0,0.93,0.85,25.91,X5K,ATH,METALS
82,UJJIVANSFB,60.00,82.03,40.0,H-SC,14.60,114750.0,-27729.0,47254.0,0.16,-19.46,41.18,13.70,163.0,-0.59,0.87,35.44,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,DMART,5201.00,-6.62,74.0,H-LC,12.84,233230.0,48749.0,21620.0,1.27,26.42,9.27,38.14,38.0,2.25,1.76,39.83,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178059.0,27146.0,30163.0,-0.08,17.99,16.94,37.97,93.0,0.90,1.34,47.84,AR,ATH,AUTO
22,COFFEEDAY,80.00,-45.67,62.0,L-SC,26.10,81542.0,-32007.0,72059.0,-0.17,-28.19,88.37,35.27,268.0,-0.44,0.62,99.58,XR,NTT,HOTELS
14,BLUESTARCO,2326.38,9.32,59.0,H-MC,4.05,188630.0,23960.0,44007.0,0.33,14.55,23.33,41.28,89.0,0.54,1.42,23.69,X40N,ATH,AC
55,NESTLEIND,1377.00,-10.64,56.0,H-LC,2.84,274973.0,9547.0,47240.0,1.09,3.60,17.18,21.40,11.0,0.20,2.07,11.19,XY25,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.74
1,25,43.18
2,50,72.77


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    36.91
LC    36.86
MC    26.21
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.18
X40      15.52
X40N     14.29
XY25      9.76
AR        9.29
XR        8.91
OX40N     7.30
X5K       2.24
X200      1.74
MH        1.67
SR        1.08
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.69
H-MC    23.33
H-SC    22.29
M-SC    13.09
M-LC     7.12
M-MC     2.56
L-SC     1.53
L-LC     1.05
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.26,-0.83,33.68
IT,13.39,-16.32,74.01
BANKS,7.34,-17.30,67.11
FINANCE,7.20,-26.21,72.74
PAINTS,5.94,-17.12,43.34
MISC,5.79,-19.68,88.29
ELECTRICAL,5.64,-10.14,70.84
AUTO,4.55,-11.87,60.54
AC,3.69,1.05,32.46


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2899771.0,23
AR,1246939.0,10
XR,1112164.0,13
X40,909290.0,10
X40N,540432.0,11
OX40N,488985.0,10
XY25,447014.0,7
SR,199620.0,2
X5K,141597.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2819212.0,27
M-SC,1507968.0,17
H-LC,1426578.0,17
H-MC,1283415.0,18
M-LC,435292.0,5
M-MC,339048.0,2
L-SC,258388.0,3
L-MC,60121.0,1
L-LC,45234.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,950184.0,6
M-SC,XY24,885217.0,7
H-SC,AR,670493.0,5
H-LC,X40,596734.0,5
H-SC,XR,532665.0,6
H-MC,XY24,492733.0,5
H-LC,AR,386008.0,2
M-MC,XY24,339048.0,2
H-MC,X40,229973.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
